# import package

In [0]:
import pandas as pd
import numpy as np
# from sklearn import cross_validation
from sklearn.datasets import load_iris

# train_test_split的作用是 把数据集分解成训练数据集，测试数据集
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import csv

# K-fold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# SVM
from sklearn.svm import SVC

## Connect Google Drive.
## This must be authorized by your Google account.
## Please note the ***file path***！

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Big Data"

os.chdir(path)
os.listdir(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['test.csv',
 'train.csv',
 'five.png.csv',
 'train-images-idx3-ubyte.gz',
 'train-images.idx3-ubyte',
 't10k-images-idx3-ubyte.gz',
 'train-labels.idx1-ubyte',
 't10k-labels-idx1-ubyte.gz',
 'train-labels-idx1-ubyte.gz',
 't10k-images.idx3-ubyte',
 't10k-labels.idx1-ubyte']

In [0]:
# Load MNIST training data
# 运行MNIST训练数据
train = pd.read_csv("train.csv")

# Pre-process MNIST data and split it into train, test data sets
# 预处理MNIST数据并将其分为 训练数据集，测试数据集
features = train.columns[1:]
X = train[features]
y = train['label']

# test_size=0.3是分解比例，分别是训练集70%和测试集30%, 如果是整数的话就是样本的数量
# random_state=0 ，决策树由于前面数据集的分解是随机的,导致由于每次生成的树都是不一样的，所以用这个参数保证每次产生的树一致
X_train, X_test, y_train, y_test = train_test_split(X/255, y, test_size=0.7, random_state=0)

# SVM
## 10-Fold CV
Tuning Hyperparameter ***C*** (Penalty factor：Tolerance for error)

In [0]:
# K-fold
Used to put the result value of each model
cv_scores = []
C_range = np.array([0.1, 0.5, 1, 5, 10, 50])
best_score = 0
best_C = 0
for i,n in enumerate(C_range):
  print('i=',i,'n=',n)
  SVM = SVC(kernel='rbf',C=n,gamma = 0.05)
  #cv：Choose the number of folds for each test, here I choose 10-fold 
  scores = cross_val_score(SVM,X_train,y_train,cv=10)   
  cv_scores.append(scores.mean())
  if (scores.mean() > best_score).all():
    best_score = scores.mean()
    best_C = n
    print("best_score: %f" %best_score)
    print("best_C %d" %best_C)
    
# Drawing
plt.plot(C_range,cv_scores,label='kernel=linear')
plt.legend(loc='best')
plt.title('Accuracy under different Penalty-Factor C')
plt.xlabel('C_range')
plt.ylabel('Accuracy')		# Select the best parameters by image
plt.annotate('Best C = ' + str(best_C) + ', Score = '+ str(round(best_score,4)), xy=(best_C, best_score), xytext=(-3,0.94), arrowprops=dict(facecolor='black', shrink=0.05))
plt.show()

i= 0 n= 0.1
best_score: 0.902846
best_C 0
i= 1 n= 0.5
best_score: 0.960000
best_C 0
i= 2 n= 1.0
best_score: 0.967143
best_C 1
i= 3 n= 5.0
best_score: 0.969286
best_C 5
i= 4 n= 10.0
i= 5 n= 50.0


In [0]:
# Train the Random Forest classifier. Fit the train data
# 训练随机森林分类器。 拟合训练的数据
clf_svm = SVC(kernel='linear',,C=best_C,gamma = 0.05)
# 选择最优的K，进行重新训练模型
best_Kfold = cross_val_score(clf_svm, X_train, y_train, cv=10, scoring='accuracy')
clf_svm.fit(X_train, y_train)




# Predict the handwritten digits in the test data
# 预测测试数据中的手写数字
y_pred_svm = clf_svm.predict(X_test)


# Measure accuracy of the prediction
# Measure accuracy of the prediction
acc_svm = accuracy_score(y_test, y_pred_svm)
print("Random Forest accuracy（set kernel=linear）: %f" % acc_svm)

# Now use the whole train set to predict the test set
clf_svm = RandomForestClassifier()
clf_svm.fit(X, y)

#Verify the trained model

In [0]:
# Predict the test data in 'test.csv' file
# TestFile="test.csv"
# 预测“test.csv”文件中的 测试数据
# TestFile = “test.csv”
# TestFile = "five.png.csv"
TestFile = "five.png.csv"
test = pd.read_csv(TestFile)
y_pred_svm = clf_svm.predict(test)
print(y_pred_svm)

In [0]:
np.set_printoptions(linewidth=200)
with open(TestFile, 'r') as csv_file:
    for didx, data in enumerate(csv.reader(csv_file)):
        if "pixel" in data[0]: continue

        # label = data[0]
        label = y_pred_svm[didx - 1]

        # The rest of columns are pixels
        # 其余列为像素
        pixels = data

        # Make those columns into a array of 8-bits pixels
        # This array will be of 1D with length 784
        # The pixel intensity values are integers from 0 to 255
        # 使这些列成为8位像素的数组
        # 该数组为一维数组，长度为784
        # 像素强度值是0到255之间的整数
        pixels = np.array(pixels, dtype='uint8')

        # Reshape the array into 28 x 28 array (2-dimensional array)
        # 将阵列重塑为28 x 28阵列（二维阵列
        pixels = pixels.reshape((28, 28))
        print(pixels)

        # Plot
        plt.title('Predicted as {label}'.format(label=label))
        plt.imshow(pixels, cmap='gray')
        plt.show()
